# Indentifying Stress Factors in Rainbow Trouts

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()
from tqdm import tqdm

## Data

In [2]:
%%time
dataPath = './data/TPM_stress_RedTrout.xlsx'

data = pd.read_excel(dataPath)


In [4]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125909 entries, 0 to 125908
Data columns (total 73 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  125908 non-null  object
 1   Control     125909 non-null  object
 2   Control.1   125909 non-null  object
 3   Control.2   125909 non-null  object
 4   Control.3   125909 non-null  object
 5   Control.4   125909 non-null  object
 6   Control.5   125909 non-null  object
 7   Control.6   125909 non-null  object
 8   Control.7   125909 non-null  object
 9   Control.8   125909 non-null  object
 10  Control.9   125909 non-null  object
 11  Control.10  125909 non-null  object
 12  Control.11  125909 non-null  object
 13  Control.12  125909 non-null  object
 14  Control.13  125909 non-null  object
 15  Control.14  125909 non-null  object
 16  Control.15  125909 non-null  object
 17  Control.16  125909 non-null  object
 18  Control.17  125909 non-null  object
 19  Control.18  125909 non-